Author: Peter Svenningsson

Email: p.o.svenningsson@tudelft.nl

Updated: 2021-04-24

# Welcome!
This is a Jypter notebook which is hosted on Google colab.
Here we can run Python code based on the interactive python (Ipython) implementation of the python language

Code is written in cell blocks which are run by pressing the arrow symbol in the top right of the cell. Any variables and functions which are defined are stored in your namespace (workspace) much like in Matlab. 

To clear the namespace, to go Runtime -> Restart runtime

# Dataset and classification task
The data has been recorded using a multi-static coherent pulsed radar configuration with three radar nodes. In this practicum we will use the data recorded by a node which transmits and recieves in V polarization. The waveform is a linearly modulated up-chip with bandwidth $45$ MHz, $18$ dBi gain, $23$ dBm transmit power and PRF $5$ kHz

The radar measures the micro-Doppler signature of a person walking towards the sensor. The classes are defined as


*   Walking
*   Walking while carrying a heavy backpack
*   Walking while carring a metal pole

The micro-Doppler signature of a three second sample is characterized by the mean and variance of the Doppler centroid and the Doppler bandwidth as you have discussed in the lectures.

The task is then to use some classification method to create optimal decision boundaries which seperates the classes and has strong predictive performance on new data. 


# Tabular data formats
In Matlab tabular data is often stored in an array.

In Python and R it is popular to store tabular data in a dataframe. You can think of a dataframe as an array with additional functionality. One of the main differences in working with a dataframe is that the columns of the array are not retrieved by an index. Rather they are retrieved by the column name. This is handy because we not have to keep track of what quantities the different columns signify.

Run the cell below to load the dataframe and print the first five samples.

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request

urllib.request.urlretrieve('https://github.com/petersvenningsson/student-resources-EE4675/blob/main/NetRad_dataframe?raw=true', 'NetRad_dataframe')
dataset_dataframe = pickle.load(open( 'NetRad_dataframe', 'rb' ))

dataset_dataframe.head()



<br> <br> <br> 


# Extracting data


---






We can extract a column as a numpy array using the column name as a key.

In [ ]:
bandwidth_mean = dataset_dataframe['Bandwidth mean'].to_numpy()
labels = dataset_dataframe['Class index'].to_numpy()

We can also extract the complete feature array using a list of the relevant column names.

In [ ]:
feature_columns = ['Bandwidth mean', 'Bandwidth std', 'Centroid mean', 'Centroid std']
feature_array = dataset_dataframe[feature_columns].to_numpy()
print(f"The shape of the feature array is {feature_array.shape[0]} rows and {feature_array.shape[1]} columns")

The shape of the feature array is 360 rows and 4 columns


Tthe spectrograms have been stored in the *Spectrogram* column. Lets define a function plot_spectrogram() which plots a spectrogram.

In [ ]:
def plot_spectrogram(sample_index):
    spectrogram = dataset_dataframe['Spectrogram'][sample_index]
    plt.imshow(spectrogram, aspect='auto', cmap='jet', origin='lower', vmin=-40, interpolation='Nearest')
    plt.colorbar()

    plt.ylabel('Doppler bin')
    plt.xlabel('Time bin')
    plt.title(f'''The class is {dataset_dataframe["Class"][sample_index]}.
        The feature Centroid std is {round(dataset_dataframe["Centroid std"].to_numpy()[sample_index],3)}''')
    plt.show()


plot_spectrogram(sample_index = 0)
plot_spectrogram(sample_index = 30)



<br> <br> <br> 




# Exercise 1: Hold-out validation


---


In predictive modelling, the usefulness of a classification model is determined by its performance on new data. Therefore the dataset is split into a training set and a test set. The model is fit to the training set and its performance is evaluate on the test set.


**Implement the function get_splits()**

The function get_splits() should
1.   Return  the indices of the training set and the test set as two lists of integers.
2.   The training and test sets should be disjoint
3. The union of the training and test sets should be equal to the set of all samples 
4. The training set should consists of 80 % of all samples

A experimental dataset typically has temporal correlations. For example, in this dataset the samples 0 to 89 are recorded from *Person 1* and samples 90 to 179 are recorded from *Person 2*. To minimize the impact of these correlations, the training and test indices should be selected randomly from the dataset.

Reference functions: random.shuffle()



In [ ]:
import random

def get_splits(dataset_dataframe):
    """ Returns training and validation set indices.

        training_indices: List of integers
        test_indices: List of integers
    """
    # Your code

    return training_indices, test_indices


#########
# TESTS # 
#########

training_indices, test_indices = get_splits(dataset_dataframe)
n_samples = len(dataset_dataframe)

assert isinstance(training_indices, list) and isinstance(test_indices, list),\
    'training_indices and test_indices should be Lists'
assert len(training_indices) == int(0.8*n_samples),\
    'The training indices have an incorrect length'
assert len(test_indices) == int(0.2*n_samples),\
    'The test indices have an incorrect length'
assert set(training_indices).intersection(set(test_indices)) == set(),\
    'The training and test sets are not disjoint'
assert set(training_indices).union(set(test_indices)) == set(range(n_samples)),\
    'Some samples are neither in the training set or test set'
assert sum(training_indices) != sum(range(int(n_samples*0.8))),\
    'The training set is ordered 0 through 288. This is incorrect - please select the training and test indices randomly'

<br> <br> <br> 




# Exercise 2: Data standardization


---


It is good practice to standardize the feature data. This entails translating the data so that the mean of each feature is equal to zero and scaling the data so that the standard deviation of each feature is one. Standardization is particularly important when working with distance based classifiers such as support-vector machines or K-nearest neighbours. This is because we need the feature dimensions to be in some sense directly comparable when working with distance metrics. Standardization is also important when working with principle component analysis (PCA) as otherwise features with high variance may domainate.

For each feature $x_i$

1.   Estimate the mean $\mu$ and standard deviation $\sigma$
2.   $x_i \leftarrow \frac{x_i-\mu}{\sigma}$

The mean $\mu$ and standard deviation $\sigma$ are estimated from the training set and the transformation $x_i = \frac{x_i-\mu}{\sigma}$ is applied to both the training set and the test set.

**Implement the function standardize_features()**

Reference functions:
np.mean()
np.std()

In [ ]:
def standardize_features(feature_array, training_indices):
    """ Standardizes the feature array based on the moments estimated
        from the training split.

        normalized_feature_array: np.array of shape (n_samples, n_features)
    """

    # Your code here

    return normalized_array


#########
# TESTS # 
#########
feature_array = dataset_dataframe[feature_columns].to_numpy()

training_indices, test_indices = get_splits(dataset_dataframe)
normalized_array = standardize_features(feature_array, training_indices)

assert np.sum(np.mean(normalized_array[training_indices,:], axis=0)) < 1e-8,\
    'The training split feature means are not close to zero'
assert np.std(np.std(normalized_array[training_indices,:], axis=0) - 1) < 1e-8,\
    'The training split feature standard deviations are not close to one'

assert np.sum(np.mean(normalized_array[test_indices,:], axis=0)) < 0.5,\
    'The test split feature means are not close to zero'
assert np.std(np.std(normalized_array[test_indices,:], axis=0) - 1) < 0.5,\
    'The test split feature standard deviations are not close to one'

<br> <br> <br> 




# Exercise 3: Performance metrics: Macro-, micro-average 


---

Many performance metrics such as recall, precision and F scores are defined for binary classification. Two alternativs to extend these metrics for the multi-class setting are micro and macro averaging. 

The performance metrics are typically defined as functions of statistics such as True Positives (TP) and false negatives (FN).
*   Macro-average: The performance metric is calculated for each class and the mean of these is taken as the macro-averaged metric.
*   Micro-average: A statistic (such as TP) are defined by the sum of the  statistic for each class.

<br>

For example, in the binary setting we define,

> $\text { Recall}=\frac{TP}{TP+FN}.$



For $C$ classes we can calculate the recall of each class $\text { Recall}_i,\quad i \in 1,\dots, C$

and the statistics for each class $TP_i, TN_i\,\quad i \in 1,\dots, C$

<br>

The micro-averaged recall is then defined as,


> $\text{ Recall}_{\text{mi}}=\frac{\sum_i^C TP_i}{\sum_i^C TP_i+\sum_i^C FN_i}.$




The macro-averaged recall is then defined as,



> $\text{ Recall}_{\text{ma}}=\frac{1}{C}\sum_i \text { Recall}_i$




<br>

*   The macro-averaged recall is sometimes referred to as balanced accuracy and it may be appropriate to use when there are large class-imbalances in the dataset. 

*   Micro-averaged recall is equal to the total accuray.

**Implement the function calculate_recall()**

Reference functions: sum(), max(), len()



In [ ]:
def calculate_recall(labels, predictions, averaging_type = 'macro'):
    """ Calculates and returns the recall metric in the multi-class setting.

        predictions: List or 1D numpy array of integers
        labels: List or 1D numpy array of integers
    """

    if averaging_type == 'macro':
    
        # Your code

        return recall

    if averaging_type == 'micro':

        # Your code

        return recall
        

#########
# TESTS # 
#########

from sklearn.metrics import recall_score

predictions = np.array([0,1,1,1,1,1,1,1,1,1,2])
labels = np.array([0,1,1,1,1,1,1,1,1,2,0])

assert np.abs(calculate_recall(labels, predictions, 'macro') - 
    recall_score(labels, predictions, average ='macro')) < 1e-2,\
    'The macro-averaging implementation is incorrect'  

assert np.abs(calculate_recall(labels, predictions, 'micro') -
    recall_score(labels, predictions, average ='micro')) < 1e-2, \
    'The micro-averaging implementation is incorrect'

<br> <br> <br> 




# Exercise 4: Classification model - Logistic regression 


---

One of the most simple classification models is logistic regression. A logistic regression model gets its name from that a linear model predicts the log-odds of a sample beloning to a specific class. Prediction of continous values like log-odds are often called regression tasks. The log-odds are named logits and are here denoted by $\underset{1 \times C}{\boldsymbol{z}}=\underset{1 \times k}{\boldsymbol{x}} \underset{\, \,k \times C}{\boldsymbol{W}}$, predicted by the weights $\boldsymbol{W}$ and $k$ features $\boldsymbol{x}$ for $C$ classes.

Since we are interested in classification, we exponentiate the logits and normalize the odds by dividing by their sum. This operation is named the softmax function,

> $P(Y=y_c) = \operatorname{softmax}(z_c)=\frac{e^{z_c}}{\sum_{i=1}^{k} e^{z_{i}}}$

where class is indicated by the $Y$ variable.

The class LogisticRegression implements a logistic regression classification model with multi-class support. The fit() method fits the model to a cross-entropy loss using gradient descent.


The input to the fit() and predict() methods is the feature array X, sometimes also referred to as the design matrix,


> $X=\left[\begin{array}{c}\boldsymbol{x}_{1} \\
\boldsymbol{x}_{2} \\
\vdots \\
\boldsymbol{x}_{n}
\end{array}\right]=\left[\begin{array}{ccccc}
x_{11} & x_{12} & x_{13} & \ldots & x_{1 k} \\
x_{21} & x_{22} & x_{23} & \ldots & x_{2 k} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
x_{m 1} & x_{n 2} & x_{n 3} & \ldots & x_{n k}
\end{array}\right]$, for $n$ data samples.


**Implement the methods: logits, softmax and predict()**

The predict() method should:


1.   Calculate the logits by calling the calculate_logits() method
2.   Calculate class probabilities by calling the softmax() method
3.   Select the most probable class as the prediction and return the predictions as a 1D array of integers



Reference functions: Matrix multiplication X @ W, np.exp(), np.argmax()






In [ ]:
from sklearn.preprocessing import OneHotEncoder

class LogisticRegression:
    def __init__(self):
        """ Initializes the weights W.
            The weights may be accessed by self.W
        """
        self.W = None

    def calculate_logits(self, X, W):
        """ Returns the logits given the features in X and weights in W.

            X: Numpy array shape (n_samples, n_features)
            W: Numpy array shape (n_features, n_classes)
            logits: Numpy array (n_samples, n_classes)
        """
        # Your code here
    
        return logits

    def softmax(self, logits):
        """ Evaluates the softmax function on the rows of logits

            logits: Numpy array shape (n_samples, n_classes)
            probabilities: Numpy array shape (n_samples, n_classes)
        """
        # Your code here

        return probabilities


    def predict(self, X):
        """ Calls the methods self.calculate_logits and self.softmax to 
        generate the class predictions for the samples in feature array X.

            predictions: Numpy array of integers shape (n_samples,)
        """

        W = self.W
        # You code here
        return predictions


    def predict_proba(self, X):
        """ Calls the methods self.calculate_logits and self.softmax to 
        generate predicted class distributions for the samples in feature array X.

            probabilities: Numpy array of integers shape (n_samples, n_classes)
        """
        
        # Your code here
        pass
        return probabilities

    
    def loss(self, X, Y_onehot, W):
        """ Calculates and returns the mean negative log likelihood
        """
        logits = self.calculate_logits(X, W)
        n_samples = X.shape[0]
        loss = 1/n_samples * (np.trace(X @ W @ Y_onehot.T) + 
                              np.sum(np.log(np.sum(np.exp(logits), axis=1))))
        return loss

    def gradient(self, X, Y_onehot, W):
        """ Calculates the gradient of the mean negative log likelihood
        """
        logits = self.calculate_logits(X, W)
        probabilities = self.softmax(logits)
        n_samples = X.shape[0]
        gradient = -1/n_samples * (X.T @ (Y_onehot - probabilities)) 
        return gradient

    def gradient_descent(self, X, Y, max_iter=10000, learning_rate=0.1):
        """ Fits the weights W by max_iter iterations of gradient descent.
        """
        onehot_encoder = OneHotEncoder(sparse=False)
        Y_onehot = onehot_encoder.fit_transform(Y.reshape(-1,1))
        W = np.zeros((X.shape[1], Y_onehot.shape[1]))
    
        for _ in range(max_iter):
            W = W - learning_rate * self.gradient(X, Y_onehot, W)

        return W

    def fit(self, X, Y):
        self.W = self.gradient_descent(X, Y)


#########
# TESTS # 
#########

model = LogisticRegression()

W_tmp = np.array([[0, 0], [1, 0], [0, 1]])
X_tmp = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

logits = model.calculate_logits(X_tmp, W_tmp)
assert logits.shape == (3, 2), 'The dimensions of the logits array are incorrect'
assert np.all(logits == np.array([[2, 3], [5, 6], [8, 9]])),\
    'The LogisticRegression.calculate_logits() method is incorrect.'

probabilities = model.softmax(logits)
assert probabilities.shape == (3, 2), ('The dimensions of the probabilities',
    'array are incorrect')
assert all(np.abs((np.sum(probabilities, axis = 1) - 1)) < 1e-5),\
    'The normalized probabilities does not sum to 1.'

model.W = W_tmp
predictions = model.predict(X_tmp)
assert predictions.shape == (3,), 'The dimensions of the predicions array are incorect'
assert all(predictions == 1), 'The predictions are incorrect.'

<br> <br> <br> 




# Exercise 5: Classification pipeline


---

In exercise 1-3 we have implemented all the steps needed to generate predictions from a classification model with holdout validation. 


**Fit and evaluate the model LogisticRegression with holdout validation**


1.   Split the dataset into a training and test set
2.   Fit the model on the training set
3.   Evaluate the model on the test set using the metric macro-averaged recall




In [ ]:
training_indices, test_indices = get_splits(dataset_dataframe)

feature_columns = ['Bandwidth mean', 'Bandwidth std', 'Centroid mean', 'Centroid std']
feature_array = dataset_dataframe[feature_columns].to_numpy()
feature_array = standardize_features(feature_array, training_indices)
labels = dataset_dataframe['Class index'].to_numpy()

model.fit(feature_array[training_indices], labels[training_indices])

# Your code here

print('The balanced accuracy is {:.2%}'.format(macro_recall))


#########
# TESTS # 
#########

assert macro_recall < 0.3, ('Something is incorrect, the model performs worse '
    'than random selection')

## Exercise 5: Questions

### 5.1
Try to run exercise 5 with and without the standardization of the features. Does feature standardization impact the performance of the model? If so, why?

Student's answer:

### 5.2
The training and test splits are randomly generated. Try to run exercise 5. several times. Does the performance change? If so, how should you characterize the performance of the model? What is the maximum and minimum performance you are able to acheive?

Student's answer:

<br> <br> <br> 




# Exercise 6: Evaluating classification models from scikit-learn


---

The LinearRegression class created in exercise 4 implements the same interface which is used by the classification models in the popular machine learning library *scikit-learn*. Namely the methods fit() and predict().

**Fit and evaluate classification models from scikit-learn with holdout validation**


*   Use the functions you have implemented above to evaluate the support-vector machine, naive Bayes and k-nearest neighbors on the provided dataset of micro-Doppler signatures
*   Evaluate the performance with and without feature standardization



In [ ]:
from sklearn.svm import SVC as SupportVectorMachine
from sklearn.naive_bayes import GaussianNB as GaussianNaiveBayes
from sklearn.neighbors import KNeighborsClassifier as KNearestNeighbors

model_SVM = SupportVectorMachine(kernel = 'rbf')
model_NB = GaussianNaiveBayes()
model_NN = KNearestNeighbors()

# Your code here

print('The balanced accuracy is {:.2%}'.format(macro_recall))

# Exercise 6: Questions
### 6.1
Evaluate the models with and without feature standardization. Which models seem to benifit from standardization?

Student's answer:

<br> <br> <br> 



# Non-obligatory exercise: Sensor fusion


---

As previously stated, the dataset has been collected using a multi-static radar network.
![](https://raw.githubusercontent.com/petersvenningsson/student-resources-EE4675/1daf71f41f1abf6ab8a45732b04e3ad36a27f2ed/radar_configuration.svg)





In [ ]:
# Load and visualize the tabular data containing Node 1, Node 2 and Node 3
urllib.request.urlretrieve('https://github.com/petersvenningsson/student-resources-EE4675/blob/main/NetRad_dataframe_fusion?raw=true', 'NetRad_dataframe')
fusion_dataframe = pickle.load(open( 'NetRad_dataframe', 'rb' ))
fusion_dataframe[358:363]

Until now we have only worked with the data from the monostatic *Node 1*. It may be beneficial to fuse the class-predictions from the three sensing nodes. One way to fuse predictions is by taking the mean of the three predicted class  probability distributions. 


**Implement sensor fusion**
1.   Implement the predict_proba() method in the LogisticRegression Class
2.   Fit one classification model for each sensing node
3.   Fuse the predictions on the test set and evaluate the results

Reference functions: np.mean()


In [ ]:
dataframe_node_1 = fusion_dataframe.loc[fusion_dataframe.Node == 'Node 1']
dataframe_node_2 = fusion_dataframe.loc[fusion_dataframe.Node == 'Node 2']
dataframe_node_3 = fusion_dataframe.loc[fusion_dataframe.Node == 'Node 3']

training_indices, test_indices = get_splits(dataframe_node_1) # This split is coherent for the three nodes
labels = dataframe_node_1['Class index'].to_numpy() # These labels are coherent for the three nodes
# Your code here

print(f'The recall for the fused prediction is {round(fused_recall,3)}')